In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, StratifiedKFold

all_predictions = []

for THIS_IDX in range(4):
    q_category = [0, 1, 2, 3, 2, 0, 3, 2, 3, 2, 0, 1, 2, 3, 0, 0, 1, 2, 1, 3, 0, 1, 2, 3, 2, 0, 2, 1, 3, 1, 0, 1, 2, 3, 1, 0, 1, 2, 3, 1, 0, 2, 0, 3, 1, 1, 2, 2, 3, 1, 0, 1, 0, 2, 1, 3, 2, 2, 3, 3]
    q_category = q_category

    main_df = pd.read_csv("/workspace/final_QIA/merged_train.csv")
    test_df = pd.read_csv("/workspace/final_QIA/phase2/test.csv")
    # questions = pd.read_excel("/workspace/COMP/QIA/Question.xlsx")

    q_list = [i + 1 for i in range(len(q_category)) if q_category[i] == THIS_IDX]

    def proc_short(x):
        if "아니다" in x:
            return 0
        elif "중립" in x:
            return 0.5
        else:
            return 1

    main_df = main_df[main_df.Q_number.isin(q_list)]
    main_df['Answer'] = main_df['Short_Answer'] + " " + main_df['Long_Answer']
    main_df['Final_Answer'] = main_df['Long_Answer']
    main_df['Short'] = main_df['Short_Answer'].apply(lambda x: proc_short(x))

    test_df = test_df[test_df.Q_number.isin(q_list)]
    test_df['Short'] = test_df['Short_Answer'].apply(lambda x: proc_short(x))

    # train = list(range(1, 241))    
    # val = list(range(241, 361))

    # rseed = 42

    # tgt = list(main_df['User_ID'].unique())
    # mbtis = [main_df[main_df.User_ID == x].iloc[0]['MBTI'] for x in tgt]
    # mbti_labels = [0 if x[THIS_IDX] in "ISTJ" else 1 for x in mbtis] 

    # kf = StratifiedKFold(n_splits=3, shuffle=True, random_state=rseed)
    # kf = [x for x in kf.split(tgt, mbti_labels)]

    # train = [tgt[i] for i in kf[0][0]] # only use the first seed. -> second seed
    # val =  [tgt[i] for i in kf[0][1]] # only use the first seed. -> second seed

    values = main_df.groupby('User_ID')['Short'].apply(lambda x: list(x)).tolist()
    mbtis =  main_df.groupby('User_ID')['MBTI'].apply(lambda x: 0 if list(x)[0][THIS_IDX] in "ISTJ" else 1).tolist()

    # train_df = main_df[main_df.User_ID.isin(train)]
    # val_df = main_df[main_df.User_ID.isin(val)]

    test_values = np.array(test_df.groupby('User_ID')['Short'].apply(lambda x: list(x)).tolist())
    
    # print(test_values.shape)
    if len(test_values.shape) == 1:
        test_values = np.reshape(test_values, (len(test_df), -1))

    for idx in range(len(values)):
        if len(values) != len(q_list):
            values[idx].extend([0 for _ in range(len(q_list) - len(values[idx]))])


    data = np.array(values)
    if len(data.shape) == 1:
        data = np.reshape(data, (360, -1))

    labels = np.array(mbtis)
    idxs = np.array(range(360))

    from sklearn.model_selection import train_test_split
    import lightgbm as lgb

    X_train, X_valid, y_train, y_valid = train_test_split(data, labels, test_size=1/5, random_state=42)

    # y_tr = labels[y_train]
    # y_val = labels[y_valid]

    # start = 0
    # end = 240
    # X_train, y_train = data[end:], labels[end:]
    # X_valid, y_valid = data[start:end], labels[start:end] 

    # Create an instance of the LGBMClassifier or LGBMRegressor
    model = lgb.LGBMClassifier()  # or lgb.LGBMRegressor() for regression
    model.fit(X_train, y_train)

    predictions = model.predict(test_values)
    all_predictions.append(predictions)

    predictions = model.predict(X_valid)
    accuracy = (predictions == y_valid).mean()  
    print("acc", accuracy)

    # b = len([ x for x in sorted(y_valid[predictions != y_val]) if x > 240] )
    # print("phase 2 acc:",  b / len([x for x in y_valid if x > 240]))
    # print()

acc 0.7638888888888888
acc 0.6666666666666666
acc 0.7083333333333334
acc 0.7638888888888888


In [12]:
### TEST ###
import csv 
fname = "lgbm_setting_b.csv"

# define the column names
fieldnames = ['User_ID', 'I/E', 'S/N', 'T/F', 'J/P']
users = test_df.User_ID.unique()

# open the CSV file for writing
with open(f"/workspace/final_QIA_{fname}", 'w', newline='') as csvfile:
    # create a writer object
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # write the header row
    writer.writeheader()

    for idx in range(120):
        
        ie = all_predictions[0][idx]
        sn = all_predictions[1][idx]
        tf = all_predictions[2][idx]
        jp = all_predictions[3][idx]

        writer.writerow({
            'User_ID': users[idx],
            'I/E': ie,
            'S/N': sn,
            'T/F': tf,
            'J/P': jp
        })

In [71]:
predictions

array([0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1,
       1, 1, 0, 0, 0, 0, 1, 0, 0, 0])